In [2]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("../../data/corp_text.csv")
df.head(5)

,Unnamed: 0,shixin_id,duty
0,1,2905,"被告上海纺友贸易有限公司应于本判决生效后十日内支付原告江苏盛虹面料有限公司货款人民币1,13..."
1,6,3650,"一、被告上海栩瑞电气设备有限公司确认尚欠原告上海宝临超亚电器有限公司货款302,725元，原..."
2,16,11617,原告上海玖峰数码科技有限公司于本判决生效之日起十日内支付被告张宁顺违法解除劳动合同赔偿金93...
3,21,15642,被执行人于2013年4月25前向申请执行人支付工资人民币5万元；被执行人于2013年5月15...
4,22,16229,一、原告上海顶天钢铁发展有限公司与被告内蒙古兴托物资有限公司于2007年1月11日签订的《内...


In [6]:
df.shape

(19699, 3)

In [9]:
df = df.sample(20)

In [10]:
import jieba

In [11]:
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

In [12]:
df["duty_cutted"] = df.duty.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/ys/98s0q77d0bl4mpqsp4lf7c740000gn/T/jieba.cache
Loading model cost 0.919 seconds.
Prefix dict has been built successfully.


In [13]:
df.duty_cutted.head()

6674              本金 加 利息 696816.74 元 以及 罚息 和 加倍 债务 利息 ^ ~
3756     一 、 被告 上海 真鹏 建设 发展 有限公司 应于 2015 年 6 月 15 日前 给付...
17828    要求 被执行人 支付 剩余 款项 4 , 000 , 000 元 ； 加倍 支付 迟延 履行...
18077    强制执行 （ 2017 ） 沪 0115 民初 47810 号 《 民事 判决书 》 第一项...
6315     被告 江苏 荣威 壁炉 制造 有限公司 于本 判决 生效 之日起 十日 内 支付 原告 上海...
Name: duty_cutted, dtype: object

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [15]:
n_features = 1000

In [16]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.duty_cutted)

In [17]:
from sklearn.decomposition import LatentDirichletAllocation

In [18]:
lda = LatentDirichletAllocation(n_components = 5,max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0)

In [19]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50,
                          max_doc_update_iter=100, max_iter=50,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [20]:
import pyLDAvis 
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.019595 -0.031730       1        1  89.548373
2      0.077260  0.007825       2        1   3.926588
1     -0.056008  0.021864       3        1   2.458560
4     -0.000794 -0.000189       4        1   2.033790
3     -0.000863  0.002231       5        1   2.032689, topic_info=  Term       Freq      Total Category  logprob  loglift
1  之日起  19.000000  19.000000  Default   9.0000   9.0000
5   民事  10.000000  10.000000  Default   8.0000   8.0000
4   期间  18.000000  18.000000  Default   7.0000   7.0000
2   十日  15.000000  15.000000  Default   6.0000   6.0000
0   义务  12.000000  12.000000  Default   5.0000   5.0000
7   负担  11.000000  11.000000  Default   4.0000   4.0000
3  受理费  10.000000  10.000000  Default   3.0000   3.0000
6   给付  12.000000  12.000000  Default   2.0000   2.0000
8   迟延  10.000000  10.000000  Default   1.0000   1.0000
1  之日起  18.375984  19.425842   Topic1  -1.7661   0.0548
4   期间  17.485091  18.522736   Topic1  -1.8158   0.0527
2   十日  14.766697  15.826081   Topic1  -1.9847   0.0411
6   给付  11.146681  12.146208   Topic1  -2.2660   0.0245
8   迟延   9.322521  10.319204   Topic1  -2.4447   0.0088
0   义务  10.250301  12.094812   Topic1  -2.3498  -0.0551
7   负担   9.302269  11.190369   Topic1  -2.4468  -0.0744
5   民事   8.419026  10.209017   Topic1  -2.5466  -0.0824
3  受理费   8.389478  10.265732   Topic1  -2.5501  -0.0914
3  受理费   1.107279  10.265732   Topic2  -1.4482   1.0105
7   负担   1.093923  11.190369   Topic2  -1.4603   0.9121
0   义务   1.089334  12.094812   Topic2  -1.4645   0.8302
5   民事   0.233090  10.209017   Topic2  -3.0064  -0.5422
8   迟延   0.231651  10.319204   Topic2  -3.0126  -0.5591
6   给付   0.239656  12.146208   Topic2  -2.9786  -0.6882
2   十日   0.252401  15.826081   Topic2  -2.9268  -0.9010
4   期间   0.235375  18.522736   Topic2  -2.9967  -1.1282
1  之日起   0.229197  19.425842   Topic2  -3.0233  -1.2024
5   民事   1.031890  10.209017   Topic3  -1.0505   1.4137
8   迟延   0.252494  10.319204   Topic3  -2.4583  -0.0048
7   负担   0.243408  11.190369   Topic3  -2.4949  -0.1225
3  受理费   0.216455  10.265732   Topic3  -2.6123  -0.1536
0   义务   0.234529  12.094812   Topic3  -2.5321  -0.2374
6   给付   0.223427  12.146208   Topic3  -2.5806  -0.2901
2   十日   0.235610  15.826081   Topic3  -2.5275  -0.5016
4   期间   0.255512  18.522736   Topic3  -2.4464  -0.5779
1  之日起   0.256946  19.425842   Topic3  -2.4408  -0.6199
3  受理费   0.278328  10.265732   Topic4  -2.1712   0.2875
7   负担   0.283053  11.190369   Topic4  -2.1543   0.2181
5   民事   0.255533  10.209017   Topic4  -2.2566   0.2076
8   迟延   0.251803  10.319204   Topic4  -2.2713   0.1822
6   给付   0.264239  12.146208   Topic4  -2.2231   0.0673
0   义务   0.248267  12.094812   Topic4  -2.2855   0.0092
2   十日   0.293710  15.826081   Topic4  -2.1174  -0.0916
1  之日起   0.296757  19.425842   Topic4  -2.1071  -0.2862
4   期间   0.268860  18.522736   Topic4  -2.2058  -0.3373
3  受理费   0.274191  10.265732   Topic5  -2.1856   0.2731
5   民事   0.269479  10.209017   Topic5  -2.2029   0.2613
8   迟延   0.260736  10.319204   Topic5  -2.2359   0.2176
7   负担   0.267715  11.190369   Topic5  -2.2095   0.1629
0   义务   0.272380  12.094812   Topic5  -2.1922   0.1025
6   给付   0.272205  12.146208   Topic5  -2.1929   0.0976
2   十日   0.277663  15.826081   Topic5  -2.1730  -0.1472
4   期间   0.277899  18.522736   Topic5  -2.1722  -0.3037
1  之日起   0.266958  19.425842   Topic5  -2.2123  -0.3915, token_table=      Topic      Freq Term
term                      
0         1  0.826801   义务
0         2  0.082680   义务
1         1  0.926601  之日起
2         1  0.947803   十日
3         1  0.779292  受理费
3         2  0.097411  受理费
4         1  0.917791   期间
5         1  0.783621   民事
5         3  0.097953   民事
6         1  0.905632   给付
7         1  0.804263   负担
7         2  0.089363   负担
8         1  0.872160   迟延, R=9, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1,

In [21]:
p = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

In [22]:
pyLDAvis.save_html(p, 'lda.html')